In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
import sys

# Web page scraping

In [2]:
# Create url templates for each kind of stats
per_g_url_template = "https://www.basketball-reference.com/leagues/NBA_{year}\
_per_game.html"
adv_url_template = "https://www.basketball-reference.com/leagues/NBA_{year}\
_advanced.html"
tot_url_template = "https://www.basketball-reference.com/leagues/NBA_{year}\
_totals.html"
per_36m_url_template = "https://www.basketball-reference.com/leagues/NBA_{year}_\
per_minute.html"
per_100p_url_template = "https://www.basketball-reference.com/leagues/NBA_{year}_\
per_poss.html"

# Put all the URL templates into a list
url_template_list = [per_g_url_template, adv_url_template, tot_url_template, 
                     per_36m_url_template, per_100p_url_template]

In [3]:
# Ask user to input start and end years
# Also checks to see if entry is a number
try:
    user_start_year = int(input("Enter start year in YYYY format: "))
except:
    print('Enter a valid 4 digit year.')
    
try:
    user_end_year = int(input("Enter end year in YYYY format: "))
except:
    print('Enter a valid 4 digit year.')

Enter start year in YYYY format: 2009
Enter end year in YYYY format: 2020


In [4]:
# Check if end year is after start year
if user_end_year >= user_start_year:
    print('Year range accepted.')
else:
    print('Year range is unacceptable.')

# Check if formats are in proper YYYY format
def check_year(user_input_year):
    if user_input_year > 999 and user_input_year < 10000: # Then check if it's 4 digits
        print('Year format accepted.')
    else:
        print('Enter a valid 4 digit year.')
        sys.exit()

# Check both entered years for formatting        
check_year(user_start_year)
check_year(user_end_year)

Year range accepted.
Year format accepted.
Year format accepted.


In [5]:
# Create empty lists to store data before appending to Dataframe
column_headers = []
player_data = []
# Create empty DataFrame for following functions to fill
df = pd.DataFrame()

In [6]:
# Empty DataFrames for each set of pages
df_adv = pd.DataFrame()
df_per_g = pd.DataFrame()
df_tot = pd.DataFrame()
df_per_36m = pd.DataFrame()
df_per_100p = pd.DataFrame()

# Create df_list of DataFrames for looping
df_list = [df_per_g, df_adv, df_tot, df_per_36m, df_per_100p]

In [7]:
# Get column headers from each page
# Assigns a new list of column headers each time this is called
def get_column_headers(soup):
    headers = []
    for th in soup.find('tr').findAll('th'):
        #print th.getText()
        headers.append(th.getText())
    #print headers # this line was for a bug check
    # Assign global variable to headers gathered by function
    return headers    
    #column_headers = [th.getText() for th in soup.find('tr').findAll('th')]

In [8]:
# Function to get player data from each page
def get_player_data(soup):
    # Temporary list within function to store data
    temp_player_data = []
    
    data_rows = soup.findAll('tr')[1:] # skip first row
    for i in range(len(data_rows)): # loop through each table row
        player_row = [] # empty list for each player row
        for td in data_rows[i].findAll('td'):
            player_row.append(td.getText()) # append separate data points
        temp_player_data.append(player_row) # append player row data
    return temp_player_data

In [9]:
# This function scrapes the stats data of one page and returns it in a DataFrame
def scrape_page(url):
    r = requests.get(url) # get the url
    soup = BeautifulSoup(r.text, 'html.parser') # Create BS object
    
    # call function to get column headers
    column_headers = get_column_headers(soup)
    
    # call function to get player data
    player_data = get_player_data(soup)
    
    # input data to DataFrame
    # Skip first value of column headers, 'Rk'
    df = pd.DataFrame(player_data, columns = column_headers[1:])
    
    return df

In [10]:
def get_season(input_year):
    first_yr = input_year - 1
    season = str(first_yr) + "-" + str(input_year)[2:]
    return season

In [11]:
# This function drops empty rows an columns, drops duplicates, and changes
# % character in columns
def gen_cleaning(df):
    # Convert values to numeric values first
    df = df.apply(pd.to_numeric, errors = 'ignore')
    
    # Drop columns with no data
    df.dropna(axis = 1, how = "all", inplace = True)
    
    # Drop rows with no data
    df.dropna(axis = 0, how = "all", inplace = True)
    
    # Remove duplicates player inputs; ie. players who were traded
    # I only kept the TOT per game season values
    #df.drop_duplicates(["Player"], keep = "first", inplace = True)
    
    # Change % symbol to _perc
    df.columns = df.columns.str.replace('%', '_perc')
    
    return df

In [12]:
# This function scrapes player data from multiple pages by start and end years
def scrape_pages(url_template, start_year, end_year, output_df):
    count = 0 
    for year in range(start_year, (end_year+1)):
        url = url_template.format(year = year) # grab URL per year
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html5lib') # Create soup item
        
        # Check to grab column headers
        if count == 0: # only append column headers once
            columns = get_column_headers(soup)
            count += 1
            
        # grab player data for each year
        player_data = get_player_data(soup)
        
        # Create temporary DataFrame first for each year
        # Duplicates are removed before putting into bigger DataFrame
        # These duplicates come from players playing on multiple teams in one season
        # This script only keeps the TOT output as Tm
        year_df = pd.DataFrame(player_data, columns = columns[1:])
        year_df.drop_duplicates(['Player'], keep = 'first', inplace = True)
        year_df.insert(0, 'Season', get_season(year)) # insert season year column
        
        # Append to big DataFrame for detailed cleaning
        output_df = output_df.append(year_df, ignore_index = True)
        
    # Do common, general cleaning practices
    output_df = gen_cleaning(output_df)
        
    return output_df

In [13]:
# Fill each DataFrame with data scraped from their respective pages
# Each print statement is a check for if any pages or functions give issues
# Added timer to check how long this was taking

start = time.time()

df_per_g = scrape_pages(per_g_url_template, user_start_year, user_end_year, df_per_g)
print("Finished per g")
df_adv = scrape_pages(adv_url_template, user_start_year, user_end_year, df_adv)
print("Finished adv")
df_tot = scrape_pages(tot_url_template, user_start_year, user_end_year, df_tot)
print("Finished tots")
df_per_36m = scrape_pages(per_36m_url_template, user_start_year, user_end_year, df_per_36m)
print("Finished per 36m")
df_per_100p = scrape_pages(per_100p_url_template, user_start_year, user_end_year, df_per_100p)
print("Finished per 100 possessions")

end = time.time()
print("Time elapsed :" +str((end - start) / 60) + " minutes")

Finished per g
Finished adv
Finished tots
Finished per 36m
Finished per 100 possessions
Time elapsed :6.32972579797109 minutes


# Data Auditing and Cleaning

In [14]:
# Check all column names to see what needs to be cleaned

print("totals")
print(list(df_tot))
print("per game")
print(list(df_per_g))
print("per 36 minutes")
print(list(df_per_36m))
print("advanced")
print(list(df_adv))
print("per 100p")
print(list(df_per_100p))

totals
['Season', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG_perc', '3P', '3PA', '3P_perc', '2P', '2PA', '2P_perc', 'eFG_perc', 'FT', 'FTA', 'FT_perc', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
per game
['Season', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG_perc', '3P', '3PA', '3P_perc', '2P', '2PA', '2P_perc', 'eFG_perc', 'FT', 'FTA', 'FT_perc', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
per 36 minutes
['Season', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG_perc', '3P', '3PA', '3P_perc', '2P', '2PA', '2P_perc', 'FT', 'FTA', 'FT_perc', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
advanced
['Season', 'Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS_perc', '3PAr', 'FTr', 'ORB_perc', 'DRB_perc', 'TRB_perc', 'AST_perc', 'STL_perc', 'BLK_perc', 'TOV_perc', 'USG_perc', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']
per 100p
['Season', 'Player', 'Pos', 'Age', 'Tm

In [15]:
# Label columns properly by adding "_tot" to the end of some column values
df_tot.columns.values[[7, 8 , 9, 11, 12, 14, 15, 18, 19]] = \
[df_tot.columns.values[[7, 8 , 9, 11, 12, 14, 15, 18, 19]][col] + "_tot" for col in range(9)]

df_tot.columns.values[21:30] = [df_tot.columns.values[21:30][col] + \
"_tot" for col in range(9)]

In [16]:
# Check column titles again
list(df_tot)

['Season',
 'Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'GS',
 'MP_tot',
 'FG_tot',
 'FGA_tot',
 'FG_perc',
 '3P_tot',
 '3PA_tot',
 '3P_perc',
 '2P_tot',
 '2PA_tot',
 '2P_perc',
 'eFG_perc',
 'FT_tot',
 'FTA_tot',
 'FT_perc',
 'ORB_tot',
 'DRB_tot',
 'TRB_tot',
 'AST_tot',
 'STL_tot',
 'BLK_tot',
 'TOV_tot',
 'PF_tot',
 'PTS_tot']

In [17]:
# drop _perc columns from per_g and per_36m
# Never mind, drop duplicates later on
# Add _per_g and _per_36m to column values
# Add _per_G to some values in df_per_g
df_per_g.columns.values[[7, 8 , 9, 11, 12, 14, 15, 18, 19]] = \
[df_per_g.columns.values[[7, 8 , 9, 11, 12, 14, 15, 18, 19]][col] + "_per_G" for col in range(9)]

df_per_g.columns.values[21:29] = [df_per_g.columns.values[21:30][col] + \
"_per_G" for col in range(8)]

# Rename PS/G to PTS_per_G
df_per_g.rename(columns={'PS/G': 'PTS_per_G'}, inplace = True)

In [18]:
df_per_36m.columns.values[[7, 8, 9, 11, 12, 14, 15, 18, 19]]

array(['MP', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FTA', 'FT_perc'],
      dtype=object)

In [19]:
# Check if proper values were changed
list(df_per_g)

['Season',
 'Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'GS',
 'MP_per_G',
 'FG_per_G',
 'FGA_per_G',
 'FG_perc',
 '3P_per_G',
 '3PA_per_G',
 '3P_perc',
 '2P_per_G',
 '2PA_per_G',
 '2P_perc',
 'eFG_perc',
 'FT_per_G',
 'FTA_per_G',
 'FT_perc',
 'ORB_per_G',
 'DRB_per_G',
 'TRB_per_G',
 'AST_per_G',
 'STL_per_G',
 'BLK_per_G',
 'TOV_per_G',
 'PF_per_G',
 'PTS']

In [20]:
# Add per_36m to its column values
df_per_36m.columns.values[[8, 9, 11, 12, 14, 15, 17, 18]] = \
[df_per_36m.columns.values[[8, 9, 11, 12, 14, 15, 17, 18]][col] + "_per_36m" \
for col in range(8)]

df_per_36m.columns.values[20:30] = [df_per_36m.columns.values[20:30][col] + "_per_36m" \
                                   for col in range(9)]

In [21]:
# Check columns were changed properly
list(df_per_36m)

['Season',
 'Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'GS',
 'MP',
 'FG_per_36m',
 'FGA_per_36m',
 'FG_perc',
 '3P_per_36m',
 '3PA_per_36m',
 '3P_perc',
 '2P_per_36m',
 '2PA_per_36m',
 '2P_perc',
 'FT_per_36m',
 'FTA_per_36m',
 'FT_perc',
 'ORB_per_36m',
 'DRB_per_36m',
 'TRB_per_36m',
 'AST_per_36m',
 'STL_per_36m',
 'BLK_per_36m',
 'TOV_per_36m',
 'PF_per_36m',
 'PTS_per_36m']

In [22]:
# Add per_100p to per 100 possession column values
list_of_changes = ['FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', \
                   'BLK', 'TOV', 'PF', 'PTS']
# Grab a list of current column names
column_values = list(df_per_100p.columns.values)

# Create a list for updated column names
updated_columns = []

# Loop through original column values to see what to replace
for value in column_values:
    if value in list_of_changes:
        updated_columns.append(value + '_per_100p')
    else:
        updated_columns.append(value)

# Update column values
df_per_100p.columns = updated_columns

In [23]:
# Check if columns are properly named
list(df_per_100p)

['Season',
 'Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'GS',
 'MP',
 'FG_per_100p',
 'FGA_per_100p',
 'FG_perc',
 '3P_per_100p',
 '3PA_per_100p',
 '3P_perc',
 '2P_per_100p',
 '2PA_per_100p',
 '2P_perc',
 'FT_per_100p',
 'FTA_per_100p',
 'FT_perc',
 'ORB_per_100p',
 'DRB_per_100p',
 'TRB_per_100p',
 'AST_per_100p',
 'STL_per_100p',
 'BLK_per_100p',
 'TOV_per_100p',
 'PF_per_100p',
 'PTS_per_100p',
 'ORtg',
 'DRtg']

In [24]:
# Find where '\xa0' columns are for removal
print(df_adv.columns[-5])
print(df_adv.columns[19])

WS/48
OWS


In [25]:
# Drop '\xa0' columns, last one first
#df_adv.drop(df_adv.columns[-5], axis = 1, inplace = True)
#df_adv.drop(df_adv.columns[19], axis = 1, inplace = True)

In [26]:
list(df_adv)

['Season',
 'Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'MP',
 'PER',
 'TS_perc',
 '3PAr',
 'FTr',
 'ORB_perc',
 'DRB_perc',
 'TRB_perc',
 'AST_perc',
 'STL_perc',
 'BLK_perc',
 'TOV_perc',
 'USG_perc',
 'OWS',
 'DWS',
 'WS',
 'WS/48',
 'OBPM',
 'DBPM',
 'BPM',
 'VORP']

In [27]:
df_adv.rename(columns = {'WS/48' : 'WS_per_48'}, inplace = True)

In [28]:
# Check to see if columns were dropped properly
list(df_adv)

['Season',
 'Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'MP',
 'PER',
 'TS_perc',
 '3PAr',
 'FTr',
 'ORB_perc',
 'DRB_perc',
 'TRB_perc',
 'AST_perc',
 'STL_perc',
 'BLK_perc',
 'TOV_perc',
 'USG_perc',
 'OWS',
 'DWS',
 'WS',
 'WS_per_48',
 'OBPM',
 'DBPM',
 'BPM',
 'VORP']

In [29]:
# Merge dataframes later on season, player name, and team
# Order of merges: tots, per_g, per_36m, adv
# DFs: df_tot, df_per_g, df_per_36m, df_adv
# Common things: Season, Player, Pos, Age, Tm, G

In [30]:
df_all = pd.merge(df_tot, df_per_g, how = "left", 
                 on = ['Season', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'FT_perc',
                      '3P_perc', '2P_perc', 'FG_perc', 'eFG_perc'])

In [31]:
df_all = pd.merge(df_all, df_per_36m, how = "left",
                 on = ['Season', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'FT_perc',
                      '3P_perc', '2P_perc', 'FG_perc'])

In [32]:
df_all = pd.merge(df_all, df_adv, how = "left",
                on = ['Season', 'Player', 'Pos', 'Age', 'Tm', 'G'])

In [33]:
df_all = pd.merge(df_all, df_per_100p, how = "left",
             on = ['Season', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 
                   'FG_perc', '3P_perc', '2P_perc', 'FT_perc'])

In [34]:
# Check columns to make sure they're all right
list(df_all)

['Season',
 'Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'GS',
 'MP_tot',
 'FG_tot',
 'FGA_tot',
 'FG_perc',
 '3P_tot',
 '3PA_tot',
 '3P_perc',
 '2P_tot',
 '2PA_tot',
 '2P_perc',
 'eFG_perc',
 'FT_tot',
 'FTA_tot',
 'FT_perc',
 'ORB_tot',
 'DRB_tot',
 'TRB_tot',
 'AST_tot',
 'STL_tot',
 'BLK_tot',
 'TOV_tot',
 'PF_tot',
 'PTS_tot',
 'MP_per_G',
 'FG_per_G',
 'FGA_per_G',
 '3P_per_G',
 '3PA_per_G',
 '2P_per_G',
 '2PA_per_G',
 'FT_per_G',
 'FTA_per_G',
 'ORB_per_G',
 'DRB_per_G',
 'TRB_per_G',
 'AST_per_G',
 'STL_per_G',
 'BLK_per_G',
 'TOV_per_G',
 'PF_per_G',
 'PTS',
 'MP_x',
 'FG_per_36m',
 'FGA_per_36m',
 '3P_per_36m',
 '3PA_per_36m',
 '2P_per_36m',
 '2PA_per_36m',
 'FT_per_36m',
 'FTA_per_36m',
 'ORB_per_36m',
 'DRB_per_36m',
 'TRB_per_36m',
 'AST_per_36m',
 'STL_per_36m',
 'BLK_per_36m',
 'TOV_per_36m',
 'PF_per_36m',
 'PTS_per_36m',
 'MP_y',
 'PER',
 'TS_perc',
 '3PAr',
 'FTr',
 'ORB_perc',
 'DRB_perc',
 'TRB_perc',
 'AST_perc',
 'STL_perc',
 'BLK_perc',
 'TOV_perc',
 'USG_perc',
 'OWS

In [35]:
# Try to drop duplicate MP columns
list(df_all.drop(['MP_x', 'MP_y'], axis = 1))

['Season',
 'Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'GS',
 'MP_tot',
 'FG_tot',
 'FGA_tot',
 'FG_perc',
 '3P_tot',
 '3PA_tot',
 '3P_perc',
 '2P_tot',
 '2PA_tot',
 '2P_perc',
 'eFG_perc',
 'FT_tot',
 'FTA_tot',
 'FT_perc',
 'ORB_tot',
 'DRB_tot',
 'TRB_tot',
 'AST_tot',
 'STL_tot',
 'BLK_tot',
 'TOV_tot',
 'PF_tot',
 'PTS_tot',
 'MP_per_G',
 'FG_per_G',
 'FGA_per_G',
 '3P_per_G',
 '3PA_per_G',
 '2P_per_G',
 '2PA_per_G',
 'FT_per_G',
 'FTA_per_G',
 'ORB_per_G',
 'DRB_per_G',
 'TRB_per_G',
 'AST_per_G',
 'STL_per_G',
 'BLK_per_G',
 'TOV_per_G',
 'PF_per_G',
 'PTS',
 'FG_per_36m',
 'FGA_per_36m',
 '3P_per_36m',
 '3PA_per_36m',
 '2P_per_36m',
 '2PA_per_36m',
 'FT_per_36m',
 'FTA_per_36m',
 'ORB_per_36m',
 'DRB_per_36m',
 'TRB_per_36m',
 'AST_per_36m',
 'STL_per_36m',
 'BLK_per_36m',
 'TOV_per_36m',
 'PF_per_36m',
 'PTS_per_36m',
 'PER',
 'TS_perc',
 '3PAr',
 'FTr',
 'ORB_perc',
 'DRB_perc',
 'TRB_perc',
 'AST_perc',
 'STL_perc',
 'BLK_perc',
 'TOV_perc',
 'USG_perc',
 'OWS',
 'DWS',
 'WS',


In [36]:
df_all.drop(['MP_x', 'MP_y'], axis = 1, inplace = True)

In [37]:
# Final check of columns
list(df_all)

['Season',
 'Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'GS',
 'MP_tot',
 'FG_tot',
 'FGA_tot',
 'FG_perc',
 '3P_tot',
 '3PA_tot',
 '3P_perc',
 '2P_tot',
 '2PA_tot',
 '2P_perc',
 'eFG_perc',
 'FT_tot',
 'FTA_tot',
 'FT_perc',
 'ORB_tot',
 'DRB_tot',
 'TRB_tot',
 'AST_tot',
 'STL_tot',
 'BLK_tot',
 'TOV_tot',
 'PF_tot',
 'PTS_tot',
 'MP_per_G',
 'FG_per_G',
 'FGA_per_G',
 '3P_per_G',
 '3PA_per_G',
 '2P_per_G',
 '2PA_per_G',
 'FT_per_G',
 'FTA_per_G',
 'ORB_per_G',
 'DRB_per_G',
 'TRB_per_G',
 'AST_per_G',
 'STL_per_G',
 'BLK_per_G',
 'TOV_per_G',
 'PF_per_G',
 'PTS',
 'FG_per_36m',
 'FGA_per_36m',
 '3P_per_36m',
 '3PA_per_36m',
 '2P_per_36m',
 '2PA_per_36m',
 'FT_per_36m',
 'FTA_per_36m',
 'ORB_per_36m',
 'DRB_per_36m',
 'TRB_per_36m',
 'AST_per_36m',
 'STL_per_36m',
 'BLK_per_36m',
 'TOV_per_36m',
 'PF_per_36m',
 'PTS_per_36m',
 'PER',
 'TS_perc',
 '3PAr',
 'FTr',
 'ORB_perc',
 'DRB_perc',
 'TRB_perc',
 'AST_perc',
 'STL_perc',
 'BLK_perc',
 'TOV_perc',
 'USG_perc',
 'OWS',
 'DWS',
 'WS',


In [38]:
# First check length of dataframe
print(len(df_all))

5830


In [39]:
# Fill Null values with 0
df_all.fillna(0, inplace = True)

In [40]:
# Address ambiguous positions and combination positions
df = df_all.groupby(['Pos'])['Pos'].nunique()
df

Pos
0        1
C        1
C-PF     1
PF       1
PF-C     1
PF-SF    1
PG       1
PG-SG    1
SF       1
SF-C     1
SF-PF    1
SF-SG    1
SG       1
SG-PF    1
SG-PG    1
SG-SF    1
Name: Pos, dtype: int64

In [41]:
# Remove where 'Pos' value is 0
df_all = df_all[df_all['Pos'] != 0]

# Then check df_all length again
print(len(df_all))

5818


In [42]:
# I think the PG-SF and C-SF positions are mistakes
# Check the value to see the player
df_all[df_all['Pos'] == 'C-SF']

,Season,Player,Pos,Age,Tm,G,GS,MP_tot,FG_tot,FGA_tot,...,DRB_per_100p,TRB_per_100p,AST_per_100p,STL_per_100p,BLK_per_100p,TOV_per_100p,PF_per_100p,PTS_per_100p,ORtg,DRtg


In [43]:
# Check Bobby Jones' actual, commonly played position
df_all[df_all['Player'] == 'Bobby Jones']

,Season,Player,Pos,Age,Tm,G,GS,MP_tot,FG_tot,FGA_tot,...,DRB_per_100p,TRB_per_100p,AST_per_100p,STL_per_100p,BLK_per_100p,TOV_per_100p,PF_per_100p,PTS_per_100p,ORtg,DRtg


In [44]:
# Create list of dual positions in DataFrame
# Create empty DataFrame to audit dual position values
column_names = list(df_all.columns.values)
dual_pos_rows = []
df_dual_pos = pd.DataFrame(columns = column_names)

In [45]:
# Gather all the dual positions by seeing which ones have a dash
for pos in df_all['Pos']:
    if "-" in pos:
        if pos not in dual_pos_rows:
            dual_pos_rows.append(pos)

In [46]:
# Append all dual position rows to a new DataFrame for auditing
for pos in dual_pos_rows:
    df_dual_pos = df_dual_pos.append(df_all[df_all['Pos'] == pos],
                                    ignore_index = True)

In [47]:
df_dual_pos
# It looks like all these players moved teams before
# Certain players have multiple positions or changed positions

,Season,Player,Pos,Age,Tm,G,GS,MP_tot,FG_tot,FGA_tot,...,DRB_per_100p,TRB_per_100p,AST_per_100p,STL_per_100p,BLK_per_100p,TOV_per_100p,PF_per_100p,PTS_per_100p,ORtg,DRtg
0,2008-09,Bobby Brown,SG-PG,24.0,TOT,68.0,2.0,931.0,141.0,360.0,...,2.1,2.9,6.5,1.2,0.2,3.5,5.2,19.9,96.0,118.0
1,2009-10,Allen Iverson*,SG-PG,34.0,TOT,28.0,24.0,865.0,136.0,316.0,...,3.8,4.8,6.8,1.1,0.1,3.9,2.9,23.3,103.0,114.0
2,2010-11,Kirk Hinrich,SG-PG,30.0,TOT,72.0,51.0,2157.0,278.0,623.0,...,3.9,4.4,6.9,1.8,0.4,2.9,4.5,17.7,107.0,111.0
3,2010-11,Jarrett Jack,SG-PG,27.0,TOT,83.0,15.0,1722.0,260.0,637.0,...,4.8,5.4,7.5,1.7,0.2,3.4,3.2,22.9,102.0,109.0
4,2012-13,Derek Fisher,SG-PG,38.0,TOT,33.0,9.0,575.0,55.0,161.0,...,3.1,3.3,4.2,1.7,0.3,2.7,4.9,15.6,97.0,108.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,2014-15,Glenn Robinson III,SG-SF,21.0,TOT,35.0,1.0,261.0,26.0,67.0,...,4.6,7.5,2.1,1.2,0.2,2.1,2.9,14.1,97.0,112.0
99,2015-16,Sonny Weems,SG-SF,29.0,TOT,43.0,0.0,499.0,42.0,110.0,...,4.5,5.2,4.8,1.0,0.1,4.0,4.6,10.5,82.0,113.0
100,2018-19,Jonathon Simmons,SG-SF,29.0,TOT,56.0,9.0,1064.0,133.0,350.0,...,4.5,5.8,5.8,1.3,0.7,3.1,4.1,16.7,95.0,111.0
101,2019-20,Allen Crabbe,SG-SF,27.0,TOT,37.0,1.0,653.0,63.0,177.0,...,4.7,5.5,2.4,1.1,0.3,1.2,3.4,12.3,97.0,117.0


In [48]:
df_dual_pos.groupby(['Player']).size().reset_index(name = 'Count').sort_values(['Count'], ascending = False).head(n=10)

,Player,Count
50,Jeff Green,2
15,Corey Brewer,2
3,Allen Iverson*,2
69,Marcus Morris,2
9,Brandon Knight,2
19,DeMarre Carroll,2
71,Maurice Harkless,1
70,Mardy Collins,1
68,Marcus Camby,1
67,Luther Head,1


In [49]:
# Check what is going on with some players with multiple positions
df_all[df_all['Player'] == 'Allen Iverson*']

,Season,Player,Pos,Age,Tm,G,GS,MP_tot,FG_tot,FGA_tot,...,DRB_per_100p,TRB_per_100p,AST_per_100p,STL_per_100p,BLK_per_100p,TOV_per_100p,PF_per_100p,PTS_per_100p,ORtg,DRtg
201,2008-09,Allen Iverson*,PG-SG,33.0,TOT,57.0,53.0,2093.0,348.0,834.0,...,3.8,4.6,7.4,2.3,0.2,3.8,2.3,26.2,103.0,109.0
661,2009-10,Allen Iverson*,SG-PG,34.0,TOT,28.0,24.0,865.0,136.0,316.0,...,3.8,4.8,6.8,1.1,0.1,3.9,2.9,23.3,103.0,114.0


In [50]:
# Find most common position for this player
df_all[df_all['Player'] == 'Allen Iverson*']\
.groupby(['Pos']).size().reset_index(name = 'Count')\
.sort_values(['Count'], ascending = False).iloc[0][0]

'PG-SG'

In [51]:
# Count of seasons played at most common position
df_all[df_all['Player'] == 'Allen Iverson*'].groupby(['Pos']).size().iloc[0]

1

In [52]:
# Use dictionary as key to replace 'Pos' values in the big DataFrame
most_common_pos = {}
# Saves a dictionary of player names with equally common positions
two_common_pos = {}
# PG, SG, SF, PF, C are 1-5, respectively
pos_key = {'PG': '1', 'SG': '2', 'SF': '3', 'PF': '4', 'C': '5'}

# Side note: This makes Tim Duncan a center

def grab_most_common_pos(df, pos_dict):
    # Loop through a dataframe and assign names and most common positions to a dictionary
    for index, row in df[['Player', 'Pos']].iterrows():
        player_name = row.iloc[0] # Assign player name to variable
        # subset position dataframe to a player
        pos_df = df[df['Player'] == player_name].groupby('Pos').size()\
        .reset_index(name = 'Count')\
        .sort_values(['Count'], ascending = False) 
        
        #dual_pos_dict = {} # Store dict of dual positions
        
        pos = pos_df.iloc[0][0] # Assign first position to variable
        second_pos = '' 
        
        # Fill in second position if it exists
        if len(pos_df) > 1:
            second_pos = pos_df.iloc[1][0]    
            
        # Check is player has a second common position
        # I don't know what to do in this situation yet
        #if pos_df.iloc[0][1] == pos_df.iloc[1][1]:
        #    dual_pos_dict['First position'] = pos
        #    dual_pos_dict['Second position'] = second_pos
        #    two_common_pos[player_name] = dual_pos_dict
        #print(player_name)
        
        if player_name not in pos_dict.keys(): # Check if name exists first
            pos_dict[player_name] = pos
    
    #return pos_dict

def clean_pos(df, pos_dict):
    # Loop through rows to check players' positions
    grab_most_common_pos(df, pos_dict)
    
    # If the most common position is a dual position, take the first one
    for name, pos in pos_dict.items():
        if '-' in pos:
            index = pos.find('-')
            pos_dict[name] = pos[:index] 
        else:
            continue
    
    # Change pos_dict values to 1-5 from key
    for key, value in pos_dict.items():
        pos_dict[key] = pos_key[value]
    
    # Return DataFrame with cleaned positions
    return df

In [53]:
# This function takes in a DataFrame and adds a new column with Rounded Position values
def assign_pos(df, pos_dict):    
    # Add a Rounded_Pos column and fill it from pos_dict
    df['Rounded_Pos'] = ''
    
    for name, pos in pos_dict.items():
        df.Rounded_Pos[df['Player'] == name] = pos

In [54]:
clean_pos(df_all, most_common_pos)

,Season,Player,Pos,Age,Tm,G,GS,MP_tot,FG_tot,FGA_tot,...,DRB_per_100p,TRB_per_100p,AST_per_100p,STL_per_100p,BLK_per_100p,TOV_per_100p,PF_per_100p,PTS_per_100p,ORtg,DRtg
0,2008-09,Alex Acker,SG,26.0,TOT,25.0,0.0,199.0,30.0,76.0,...,4.2,6.3,3.2,1.3,1.1,1.8,2.4,19.0,94.0,114.0
1,2008-09,Hassan Adams,SG,24.0,TOR,12.0,0.0,52.0,4.0,13.0,...,6.0,7.0,1.0,1.0,1.0,4.0,4.0,11.0,63.0,111.0
2,2008-09,Arron Afflalo,SG,23.0,DET,74.0,8.0,1234.0,131.0,300.0,...,4.7,6.0,2.0,1.3,0.6,1.9,6.5,16.3,107.0,110.0
3,2008-09,Maurice Ager,SG,24.0,NJN,20.0,0.0,97.0,15.0,43.0,...,3.9,5.5,1.7,0.6,1.1,2.2,9.4,18.7,77.0,115.0
4,2008-09,Blake Ahearn,PG,24.0,SAS,3.0,0.0,19.0,2.0,6.0,...,2.9,2.9,5.7,2.9,0.0,2.9,8.6,22.9,113.0,104.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5825,2019-20,Trae Young,PG,21.0,ATL,60.0,60.0,2120.0,546.0,1249.0,...,4.9,5.6,12.3,1.4,0.2,6.4,2.3,39.1,114.0,117.0
5826,2019-20,Cody Zeller,C,27.0,CHO,58.0,39.0,1341.0,251.0,479.0,...,9.4,15.4,3.3,1.5,0.9,2.8,5.2,24.0,116.0,111.0
5827,2019-20,Tyler Zeller,C,30.0,SAS,2.0,0.0,4.0,1.0,4.0,...,11.9,47.8,0.0,0.0,0.0,0.0,0.0,23.9,95.0,116.0
5828,2019-20,Ante Žižić,C,23.0,CLE,22.0,0.0,221.0,41.0,72.0,...,10.6,14.5,1.3,1.5,1.1,2.2,5.9,21.1,116.0,113.0


In [55]:
assign_pos(df_all, most_common_pos)

<ipython-input-53-394a4ae3b7b6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Rounded_Pos[df['Player'] == name] = pos


# Write to csv file for use

In [56]:
# Create a DataFrame with top 25 single season scorers 
#df_top_25_scorers = df_all.sort_values('PTS_per_G', ascending = False).head(n=25)

# Create a DataFrame with top 50 single season scorers 
#df_top_50_scorers = df_all.sort_values('PTS_per_G', ascending = False).head(n=50)

In [57]:
# Write to CSV files and DONE!
file_name = 'player_data_' + str(user_start_year) + '-' + str(user_end_year) + '.csv'
#print(file_name)
df_all.to_csv(file_name, encoding = 'utf-8', index = False)

In [58]:
#df_top_50_scorers.to_csv("bref_1981_2017_top_50_season_scorers.csv", encoding = "utf-8", index = False)